In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('Banco de Súmulas - Sumulas.csv')

In [16]:
# choose class
data = data[data['Classe Vela'] == 'IQFOIL Fem.']

In [17]:
# create dictionary to map competitor names to unique indices
competitor_to_index = {competitor: idx for idx, competitor in enumerate(data['Nome Competidor'].unique())}

# create matrix of size (n_players, n_players) with zeros
n_players = len(competitor_to_index)
matrix_alpha = np.zeros((n_players, n_players))
matrix_beta = np.zeros((n_players, n_players))

# iterate through competitions
for competition in data['Nome Competição'].unique():
    # get data for this competition
    data_competition = data[data['Nome Competição'] == competition]
    data_competition = data_competition.drop_duplicates(subset='Nome Competidor', keep='first')

    # get list of competitors in this competition
    competitors = data_competition['Nome Competidor'].unique()
    # get number of competitors in this competition
    n_players = len(competitors)
    
    # iterate through competitors in this competition setting (i, j) = 1 if i beats j in this competition
    # get i and j from the dictionary
    for i in range(n_players):
        for j in range(n_players):
            competitor_i = competitors[i]
            competitor_j = competitors[j]
            posicao_i = data_competition[(data_competition['Nome Competidor'] == competitor_i)]['Posição Geral'].values[0]
            posicao_j = data_competition[(data_competition['Nome Competidor'] == competitor_j)]['Posição Geral'].values[0]

            # get index of competitor i and j in the matrix
            index_i = competitor_to_index[competitor_i]
            index_j = competitor_to_index[competitor_j]

            if posicao_i < posicao_j:
                # update the cumulative matrix
                matrix_alpha[index_i][index_j] += 1
                # update matrix beta with difference in points
                matrix_beta[index_i][index_j] += (posicao_j - posicao_i)/(posicao_i + posicao_j)

In [18]:
matrix_alpha.shape

(179, 179)

In [15]:
# write_matrix = pd.DataFrame(matrix_beta)

# # columns and index are the names of the competitors
# write_matrix.columns = competitor_to_index.keys()
# write_matrix.index = competitor_to_index.keys()

# write_matrix


,Hélène NOESMOEN,Emma WILSON,Maja DZIARNOWSKA,Islay WATSON,Sara WENNEKES,Marta MAGGETTI,Mariana AGUILAR CHAVEZ PEON,Pilar LAMADRID TRUEBA,Daniela PELEG,Shahar TIBI,...,Lilian de GEUS,YAN Zheng,LU Yunxiu,HUANG Xianting,Pilar GONZALEZ de la MADRID TRUEBA,Nicole van der VELDEN,SUNAGA Yuki,DU Jie,ZHENG Manjia,Helene NOESMOEN
Hélène NOESMOEN,0.000000,0.333333,1.169856,1.155556,1.317647,1.600000,1.209459,1.208547,1.814049,1.059728,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
Emma WILSON,1.539986,0.000000,2.868665,2.994067,2.981719,2.617409,2.539683,1.926797,3.118094,2.107143,...,0.750000,0.454545,0.872340,0.700000,0.600000,0.739130,0.869565,0.863636,0.866667,0.25
Maja DZIARNOWSKA,0.200000,0.000000,0.000000,0.285714,0.671456,0.565591,0.951282,0.597403,1.144666,0.578462,...,0.050000,0.000000,0.396825,0.000000,0.000000,0.025641,0.387097,0.366667,0.377049,0.00
Islay WATSON,0.264957,0.000000,0.234483,0.000000,0.682126,0.651282,0.935584,0.666667,1.150651,0.666667,...,0.000000,0.000000,0.313433,0.000000,0.000000,0.000000,0.303030,0.281250,0.292308,0.00
Sara WENNEKES,0.000000,0.000000,0.571014,0.625668,0.000000,0.515152,0.452381,0.263027,1.337232,0.333333,...,0.312500,0.000000,0.600000,0.214286,0.043478,0.290323,0.592593,0.576923,0.584906,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Nicole van der VELDEN,0.000000,0.000000,0.000000,0.069767,0.000000,0.000000,0.000000,0.000000,0.183673,0.000000,...,0.024390,0.000000,0.375000,0.000000,0.000000,0.000000,0.365079,0.344262,0.354839,0.00
SUNAGA Yuki,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
DU Jie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.035294,0.000000,0.000000,0.000000,0.023810,0.000000,0.012048,0.00
ZHENG Manjia,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.023256,0.000000,0.000000,0.000000,0.011765,0.000000,0.000000,0.00


In [19]:
# Keener's method
# compute the sum of each row
sums = matrix_alpha.sum(axis=1)
# divide each row by its sum
matrix_alpha = matrix_alpha / sums[:, np.newaxis]



C:\Users\Carol Erthal\AppData\Local\Temp\ipykernel_2892\1943534568.py:5: RuntimeWarning: invalid value encountered in divide
  matrix_alpha = matrix_alpha / sums[:, np.newaxis]
